In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load the webpage into a document

loader = WebBaseLoader(
    web_paths=("https://www.mementobelfast.com/plantcareinfo/",),
)
docs = loader.load()

# Split the document into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# Embed the contents of each document split and insert these embeddings into a vector store

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Define a retriever that can return relevant documents given a string query

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

question = input("My question about plants...")

result = rag_chain.invoke({"input": question})

print(f"Question: {question}")
print(f"Answer: {result["answer"]}")

contextDocs = result["context"]
print(f"Context: {len(contextDocs)} documents")
print(f"Example context: {contextDocs[0].page_content[:100]}...")
